In [1623]:
import nupack
from nupack import *
from collections import deque
import pandas as pd
import gspread
import RNA
from copy import copy, deepcopy
#Change below from 20 to 'None' if you want to see full display
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
import numpy as np
import yaml

In [1624]:
model1 = Model()
#Model parameters go here

salt_concentrations_list = [0,0.01,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
#salt_concentrations_list = [0, 0.01]
csv_names = ['(0.0) Molarity-Coxsackievirus-Data.csv',
             '(0.01) Molarity-Coxsackievirus-Data.csv',
             '(0.1) Molarity-Coxsackievirus-Data.csv',
             '(0.2) Molarity-Coxsackievirus-Data.csv',
             '(0.3) Molarity-Coxsackievirus-Data.csv',
             '(0.4) Molarity-Coxsackievirus-Data.csv',
             '(0.5) Molarity-Coxsackievirus-Data.csv',
             '(0.6) Molarity-Coxsackievirus-Data.csv',
             '(0.7) Molarity-Coxsackievirus-Data.csv',
             '(0.8) Molarity-Coxsackievirus-Data.csv',
             '(0.9) Molarity-Coxsackievirus-Data.csv',
             '(1.0) Molarity-Coxsackievirus-Data.csv']
temp = 25.0
md = RNA.md()
md.temperature = temp

#md_parameters = len(salt_molarity)
#end of parameters

sheet_link = 'https://docs.google.com/spreadsheets/d/1idf3GHur5Eneef_vnwpGD-dn_dhoEgqx/edit#gid=1170226371'
sheet_rows = 34 #how many rows we want to use
f_part = sheet_link.split('/edit#gid')[0]
csv_link = f_part + '/export?format=csv'
data = pd.read_csv(csv_link)
df = pd.read_csv(csv_link, usecols=['toehold_switch_sequence','target_sequence', 'location'], nrows=sheet_rows)
# df.insert(len(df.columns), 'switch_energy', 1.0)
# df.insert(len(df.columns), 'target_energy', 1.0)
# df.insert(len(df.columns), 'switch_gc_count', -1)
# df.insert(len(df.columns), 'target_gc_count',-1)
# df.insert(len(df.columns), 'ensemble_defect_switch', 0)
# df.insert(len(df.columns), 'ensemble_defect_target', 0)
df.columns = df.columns.str.replace('location', 'target_location')

preliminary_data_names = ['switch_structure', 'target_structure', 'switch_energy', 'target_energy', 'switch_gc', 'target_gc', 'switch_ensemble_defect', 'target_ensemble_defect', 'switch_single_strandedness', 'target_single_strandedness']
old_data_names = ['salt_molarity', 'temperature (Celsius)', 'switch_row', 'target_row', 'target_location', 'toehold_switch_sequence', 'target_sequence', 'switch_mfe_structure', 'target_mfe_structure', 'switch_energy','switch_gc', 'switch_ensemble_defect', 'switch_single_strandedness', 'target_energy', 'target_gc', 'target_ensemble_defect', 'target_single_strandedness', 'valid_loop', 'loop_sequence', 'loop_structure', 'loop_energy', 'complex_nucelotides', 'complex_structure', 'complex_visualization_structure', 'complex_energy', 'complex_gc', 'complex_ensemble_defect', 'complex_single_strandedness']
finished_data_names = ['salt_molarity', 'temperature (Celsius)', 'switch_row', 'target_row', 'target_location', 'toehold_switch_sequence', 'target_sequence', 'complex_nucelotides', 'switch_structure', 'target_structure', 'complex_structure', 'complex_visualization_structure', 'switch_energy', 'target_energy', 'complex_energy', 'switch_gc', 'target_gc', 'complex_gc', 'switch_ensemble_defect', 'target_ensemble_defect', 'complex_ensemble_defect', 'switch_single_strandedness', 'target_single_strandedness', 'complex_single_strandedness', 'valid_loop', 'loop_sequence', 'loop_structure', 'loop_energy']

In [1625]:
#computes ammounts of gc in sequence
def oneComputeGCtotal(seq):
    gc_content = 0
    total_bases = len(seq)
    idx = 0
    while idx < len(seq):
        seq_idx = seq[idx]
        if seq_idx == 'C' or seq_idx == 'G':
            gc_content += 1
        idx += 1
    gc_cont_perc = float(gc_content) / float(total_bases)
    return gc_cont_perc

In [1626]:
def find_single_strandedness(base_pair_probability_matrix): #This was taken from Andrew's code, for some reason it's not working as intended... possible that diagonals means something different in Vienna
    assert base_pair_probability_matrix.ndim == 2, "matrix must be two dimensional"
    assert base_pair_probability_matrix.shape[0] == base_pair_probability_matrix.shape[1], "matrix must be square"

    sum_of_diagonal = np.sum(np.diag(base_pair_probability_matrix))
    length_of_sequence = base_pair_probability_matrix.shape[0]
    return sum_of_diagonal/length_of_sequence

In [1627]:
def compute_preliminary_data(row): #This is the everything method for the panda applys
    switch_seq = row['toehold_switch_sequence']
    target_seq = row['target_sequence']
    switch_fc = RNA.fold_compound(switch_seq, md) #Fold compound takes a sequence and a model, we run methods on it to compute the things we want
    target_fc = RNA.fold_compound(target_seq, md)
    (switch_struct, switch_energy) = switch_fc.mfe() #mfe returns min free energy structure + energy
    (target_struct, target_energy) = target_fc.mfe()

    #Compute the gc count
    switch_gc = oneComputeGCtotal(switch_seq)
    target_gc = oneComputeGCtotal(target_seq)

    #we can use these for later, but we need to calculate BP probs and ensemble defect with pf first, which returns these results. Might as well store them for later
    #It basically computes the partiton function of a sequence (I don't exactly know what that means but it's needed)
    (switch_propensity_struct, switch_ensemble_energy) = switch_fc.pf()
    (target_propensity_struct, target_ensemble_energy) = target_fc.pf()

    #TODO: for later, figure out if we use MFE structure or ensemble structure for ensemble defect prediction
    #Computes ensemble defect (higher percent is bad)
    switch_ensemble_defect = switch_fc.ensemble_defect(switch_struct)
    target_ensemble_defect = target_fc.ensemble_defect(target_struct)

    #Below is to calculate base pair probability matrix and single strandedness OLD CODE
    switch_bpp = switch_fc.bpp() #Gives us base bair probability
    np_switch = np.asarray(switch_bpp)
    switch_single_strandedness = find_single_strandedness(np_switch)
    target_bpp = target_fc.bpp()
    np_target = np.asarray(target_bpp)
    target_single_strandedness = find_single_strandedness(np_target)
    
    #print(preliminary_data_names)
    return [switch_struct,target_struct, switch_energy, target_energy, switch_gc, target_gc, switch_ensemble_defect, target_ensemble_defect, switch_single_strandedness, target_single_strandedness]

In [1628]:
df[preliminary_data_names] = df.apply(compute_preliminary_data, axis=1, result_type='expand')

In [1629]:
df.head()

,target_location,toehold_switch_sequence,target_sequence,switch_structure,target_structure,switch_energy,target_energy,switch_gc,target_gc,switch_ensemble_defect,target_ensemble_defect,switch_single_strandedness,target_single_strandedness
0,6946,GGGGAAGAGUCUAUUGAGCUAGUUGGUAGUCCUAUACAGAAACAGA...,AGGACTACCAACTAGCTCAATAGACTCTTC,........(((.......(((((((((.(((((((((............,.((....))..(((......))).......,-36.759998,-2.66,0.464646,0.433333,0.100807,0.133066,0.0,0.0
1,6805,GGGGUUUCAUUUUAUUCCUAUACUGGCUGCUUAAUACAGAAACAGA...,TAAGCAGCCAGTATAGGAATAAAATGAAAC,(((((........)))))(((((((((((.(((((((............,.......((......)).............,-34.180000,-1.87,0.414141,0.333333,0.051809,0.026669,0.0,0.0
2,6804,GGGUUUCAUUUUAUUCCUAUACUGGCUGCUUAUAUACAGAAACAGA...,ATAAGCAGCCAGTATAGGAATAAAATGAAA,(((...........)))((((((((((..((((((((............,........((......))............,-32.110001,-1.87,0.404040,0.300000,0.125341,0.028123,0.0,0.0
3,6771,GGGGACAAUUGAGAGAUUGUUACCAUAUAGCUAAUACAGAAACAGA...,TAGCTATATGGTAACAATCTCTCAATTGTC,........(((..((.(((((((((((((.(((((((............,..(((....))).(((((......))))).,-34.939999,-3.57,0.414141,0.333333,0.048487,0.166705,0.0,0.0
4,6712,GGGCUAUUAUAUAUCCCUUUGUUGGGUUUAUACAUACAGAAACAGA...,GTATAAACCCAACAAAGGGATATATAATAG,(((.(((((((((((((((((((..((........))...))))))...,(((((..(((......))).))))).....,-43.279999,-7.34,0.414141,0.300000,0.039088,0.072352,0.0,0.0


In [1630]:
def compute_structure(seq):
    return RNA.fold_compound(seq,md).mfe()[0]

In [1631]:
def compute_defect(sequence,structure):
    fc = RNA.fold_compound(sequence,md)
    return fc.ensemble_defect(structure)


In [1632]:
# Really bad old code (then again my code is always bad)
#Set up of the table
# #Note: making seperate functions for each purpose as apply is optimized compared to for loops with pandas apparently. Reason why I have functions that compute similar things
# df['switch_mfe_structure'] = df['toehold_switch_sequence'].apply(compute_structure)
# df['target_mfe_structure'] = df['target_sequence'].apply(compute_structure)
# df['switch_gc_count'] = df['toehold_switch_sequence'].apply(oneComputeGCtotal)
# df['target_gc_count'] = df['target_sequence'].apply(oneComputeGCtotal)
# df['switch_energy'] = df['toehold_switch_sequence'].apply(computeEnergy)
# df['target_energy'] = df['target_sequence'].apply(computeEnergy)
# df['ensemble_defect_switch'] = df.apply(lambda x: compute_defect(x['toehold_switch_sequence'], x['switch_mfe_structure']), axis=1)
# df.head()

In [1633]:
def test_valid_notation(d): #just sees that every opening parenthesis has a closing parethesis (i.e is valid notation)
    q = deque()
    for let in d:
        if let == '(':
            q.append(let)
        elif let == ')':
            if (len(q)==0):
                return 1 #if extra closed parethesis
            q.pop()
    if(len(q)==0):
        return 0 #if valid
    return 2 #if extra open parethesis

In [1634]:
#NOTE: actually figure out which nueclotides are the stem loop region, this seems off
def stem_loop_eval(s): #Stem loop region is 35-48 nucleotide inclusive
    start = 35
    end = 49
    strand_sub = s[start:end]
    ind = 1
    limit = 3 #Limit is 3 because seems like a good num i suppose
    result = test_valid_notation(strand_sub)
    #print(strand_sub)
    while result!=0 and ind<=limit: #there might be off by 1 errors, so we are gonna add up to the limit that many nueclotides to the sequence to try to get a valid sequence
        if result==1:
            start-=1
        else:
            end+=1
        strand_sub = s[start:end]
        #print(strand_sub) 
        result = test_valid_notation(strand_sub)
        #print(ind)
        ind+=1
    if result == 0:
        return True, start, end
    return False, -1, -1
        

In [1635]:
#These are the col names of the table being filled in, later the order of the columns will be changed according to the 'finished data names up top
col_names = ['salt_molarity', 'temperature (Celsius)', 'switch_row','target_row', 'target_location', 'toehold_switch_sequence', 'target_sequence'] + preliminary_data_names[0:4] + ['valid_loop', 'loop_sequence', 'loop_structure','loop_energy','complex_nucelotides','complex_structure', 'complex_visualization_structure','complex_energy','complex_gc'] + preliminary_data_names[4:8] + ['complex_ensemble_defect'] + preliminary_data_names[8:] + ['complex_single_strandedness']
print(col_names)
len(col_names)

['salt_molarity', 'temperature (Celsius)', 'switch_row', 'target_row', 'target_location', 'toehold_switch_sequence', 'target_sequence', 'switch_structure', 'target_structure', 'switch_energy', 'target_energy', 'valid_loop', 'loop_sequence', 'loop_structure', 'loop_energy', 'complex_nucelotides', 'complex_structure', 'complex_visualization_structure', 'complex_energy', 'complex_gc', 'switch_gc', 'target_gc', 'switch_ensemble_defect', 'target_ensemble_defect', 'complex_ensemble_defect', 'switch_single_strandedness', 'target_single_strandedness', 'complex_single_strandedness']


28

In [1636]:
def fill_matrix(arr_loc, arr, data):
    for k in range(len(col_names)):
        # print(len(arr[arr_loc]))
        #print(len(data))
        arr[arr_loc][k] = data[k]  
    

In [1637]:
#TODO: Make multiple csvs corresponding to the salt molarities
ribo_size = len(df.dropna().toehold_switch_sequence)
switch_size = len(df.dropna().target_sequence)
vArr = [[0 for i in range (len(col_names))] for j in range (ribo_size*switch_size)] #fix later lol
test = deepcopy(vArr)
full_table = [[] for i in range(1)]
#0 is salt
#1 is temperature
#2 is switch row
#3 is target row
#4 is target location
#5 is toehold switch sequence
#6 is target sequence
#7 is switch mfe struct
#8 is target mfe struct
#9 is switch energy
#10 is target energy
#11 is valid stem loop
#12 is stem loop sequence
#13 is stem loop structure
#14 is stem loop energy
#15 is complex nucelotides
#16 is complex structure
#17 is complex visualization
#18 is complex energy
#19 is complex gc count
#20 is switch gc count
#21 is target gc count
#22 is switch ensemble defect
#23 is target ensemble defect
#24 is complex ensemble defect
#25 is switch single strandness
#26 is target single strandness
#27 is complex single strandness
for i in range(len(salt_concentrations_list)):
    md.salt = salt_molarity = salt_concentrations_list[i]
    
    #print(concentration)
    df[preliminary_data_names] = df.apply(compute_preliminary_data, axis=1, result_type='expand')
    for x in range(ribo_size):
        switch_struct = df.at[x,'switch_structure']
        switch_seq = df.at[x,'toehold_switch_sequence']

        for y in range(switch_size):
            data_list = []
            target_seq = df.at[y,'target_sequence']
            target_struct = df.at[y,'target_structure']
            arr_loc = x * switch_size + y

            #data 0-10
            #adding salt, temperature, switch row, target row, target location, toehold switch sequence, target sequence, switch mfe structure, target mfe structure, switch energy, and target energy to list
            data_list.extend([salt_molarity, temp, x, y, df.at[y, 'target_location'], switch_seq, target_seq, switch_struct, target_struct, df.at[x,'switch_energy'], df.at[y,'target_energy']])

            #This is scuffed, there should be a better way but this will do for now
            #This evaluates the stem loop and its free energy, if it's valid
            outcome = stem_loop_eval(switch_struct)
            #Add if the loop is valid (#11)
            validity = outcome[0]
            data_list.append(validity)

            #now add the loop sequence, structure, and energy if applicable (#12-14)

            if(validity==False): #if not valid from stem loop test 
                data_list.extend(['N/A', 'N/A', 'N/A'])
            else:
                #we use substrings of the switches sequence and structure to get the loop structure and sequence
                sub_seq = switch_seq[outcome[1]:outcome[2]]
                sub_struct = switch_struct[outcome[1]:outcome[2]]
                #print(sub_struct)
                data_list.extend([sub_seq, sub_struct, RNA.fold_compound(sub_seq,md).mfe()[1]])
            
            #Add the complex sequence (#15)
            data_list.append(target_seq+'+'+switch_seq)

            # NOTE SOME OF THE SEQUENCES HAVE T's IN THEM INSTEAD OF U's... WE GOTTA REPLACE THEM, Orignial is still listed in table
            new_switch_seq = switch_seq.replace('T','U')

            fc = RNA.fold_compound(target_seq+'&'+new_switch_seq, md)
            (v_complex_struct, v_complex_mfe) = fc.mfe()
            #Add the structure, visualization structure, mfe energy, complex gc count, switch gc count, and target gc count (#16-21)
            data_list.extend([v_complex_struct, v_complex_struct.replace('+','&'), v_complex_mfe, oneComputeGCtotal(target_seq+switch_seq),df.at[x,'switch_gc'] ,df.at[y, 'target_gc']])
            
            #ensemble defect/parititon function calculations
            #We add the switch, target, and complex ensemble defect here (#22-24)
            (complex_propensity, complex_ensemble_energy) = fc.pf()
            data_list.extend([df.at[x, 'switch_ensemble_defect'], df.at[y, 'target_ensemble_defect'], fc.ensemble_defect(v_complex_struct)])

            #Calculate and add the switch, target, and complex single strandedness (#24-27)
            complex_bpp = fc.bpp()
            np_complex = np.asarray(complex_bpp)
            data_list.extend([df.at[x, 'switch_single_strandedness'],df.at[y, 'target_single_strandedness'], find_single_strandedness(np_complex)])

            fill_matrix(arr_loc, vArr, data_list)
            fill_matrix(arr_loc, test, data_list)
            full_table.append(data_list)
    concentration_data = pd.DataFrame(vArr)
    concentration_data.columns = col_names
    concentration_data = concentration_data[finished_data_names]
    concentration_data.to_csv(csv_names[i])
    # print("this is the array")
    # display(pd.DataFrame(vArr))
    # print('this is the full_table')
    # display(pd.DataFrame(full_table))

        

full_table = full_table[1:] #To get rid of the first empty row
#full_table.insert(0, col_names)
#print(vArr)
results = pd.DataFrame(full_table)
testing = pd.DataFrame(test)

In [1638]:
# #TODO: Append data to a list and then assign it
# #ALSO look at why tf switch gc and target was changed
# ribo_size = len(df.dropna().toehold_switch_sequence)
# switch_size = len(df.dropna().target_sequence)
# vArr = [[0 for i in range (len(col_names))] for j in range (ribo_size*switch_size)] #fix later lol
# test = deepcopy(vArr)

# #0 is switch row
# #1 is target row
# #2 is target location
# #3 is toehold switch sequence
# #4 is target sequence
# #5 is switch mfe struct
# #6 is target mfe struct
# #7 is switch energy
# #8 is target energy
# #9 is valid stem loop
# #10 is stem loop sequence
# #11 is stem loop structure
# #12 is stem loop energy
# #13 is complex nucelotides
# #14 is complex structure
# #15 is complex visualization
# #16 is complex energy
# #17 is complex gc count
# #18 is switch gc count
# #19 is target gc count
# #20 is switch ensemble defect
# #21 is target ensemble defect
# #22 is complex ensemble defect
# #23 is switch single strandness
# #24 is target single strandness
# #25 is complex single strandness

# for x in range(ribo_size):
#     switch_struct = df.at[x,'switch_structure']
#     switch_seq = df.at[x,'toehold_switch_sequence']

#     for y in range(switch_size):
#         data_list = []
#         target_seq = df.at[y,'target_sequence']
#         target_struct = df.at[y,'target_structure']
#         arr_loc = x * switch_size + y

#         #data 0-8
#         #adding switch row, target row, target location, toehold switch sequence, target sequence, switch mfe structure, target mfe structure, switch energy, and target energy to list
#         data_list.extend([x, y, df.at[y, 'target_location'], switch_seq, target_seq, switch_struct, target_struct, df.at[x,'switch_energy'], df.at[y,'target_energy']])

#         #This is scuffed, there should be a better way but this will do for now
#         #This evaluates the stem loop and its free energy, if it's valid
#         outcome = stem_loop_eval(switch_struct)
#         #Add if the loop is valid (#9)
#         validity = outcome[0]
#         data_list.append(validity)

#         #now add the loop sequence, structure, and energy if applicable (#10-12)

#         if(validity==False): #if not valid from stem loop test 
#             data_list.extend(['N/A', 'N/A', 'N/A'])
#         else:
#             #we use substrings of the switches sequence and structure to get the loop structure and sequence
#             sub_seq = switch_seq[outcome[1]:outcome[2]]
#             sub_struct = switch_struct[outcome[1]:outcome[2]]
#             #print(sub_struct)
#             data_list.extend([sub_seq, sub_struct, RNA.fold_compound(sub_seq,md).mfe()[1]])
        
#         #Add the complex sequence (#13)
#         data_list.append(target_seq+'+'+switch_seq)

#         # NOTE SOME OF THE SEQUENCES HAVE T's IN THEM INSTEAD OF U's... WE GOTTA REPLACE THEM, Orignial is still listed in table
#         new_switch_seq = switch_seq.replace('T','U')

#         fc = RNA.fold_compound(target_seq+'&'+new_switch_seq, md)
#         (v_complex_struct, v_complex_mfe) = fc.mfe()
#         #Add the structure, visualization structure, mfe energy, complex gc count, switch gc count, and target gc count (#14-19)
#         data_list.extend([v_complex_struct, v_complex_struct.replace('+','&'), v_complex_mfe, oneComputeGCtotal(target_seq+switch_seq),df.at[x,'switch_gc'] ,df.at[y, 'target_gc']])
        
#         #ensemble defect/parititon function calculations
#         #We add the switch, target, and complex ensemble defect here (#20-22)
#         (complex_propensity, complex_ensemble_energy) = fc.pf()
#         data_list.extend([df.at[x, 'switch_ensemble_defect'], df.at[y, 'target_ensemble_defect'], fc.ensemble_defect(v_complex_struct)])

#         #Calculate and add the switch, target, and complex single strandedness (#23-25)
#         complex_bpp = fc.bpp()
#         np_complex = np.asarray(complex_bpp)
#         data_list.extend([df.at[x, 'switch_single_strandedness'],df.at[y, 'target_single_strandedness'], find_single_strandedness(np_complex)])

#         fill_table(arr_loc, vArr, data_list)
#         fill_table(arr_loc, test, data_list)
        

        

# vArr.insert(0, col_names)
# test.insert(0, col_names)
# #print(vArr)
# results = pd.DataFrame(vArr)
# testing = pd.DataFrame(test)

In [1639]:
#display(results)
results.columns = col_names
results = results[finished_data_names]
#display(results)

In [1640]:
# testing = testing.rename(columns=testing.iloc[0])
# testing = testing.drop(0)
# testing = testing.reset_index(drop = True)
testing.columns = col_names
testing = testing[finished_data_names]
# testing.insert(0, 'salt_molarity', salt_molarity)
# testing.insert(1, 'temperature (Celsius)', temp)

In [1641]:
# original_cols = results.columns.to_list()
# print(original_cols)



In [1642]:
display(results)


,salt_molarity,temperature (Celsius),switch_row,target_row,target_location,toehold_switch_sequence,target_sequence,complex_nucelotides,switch_structure,target_structure,complex_structure,complex_visualization_structure,switch_energy,target_energy,complex_energy,switch_gc,target_gc,complex_gc,switch_ensemble_defect,target_ensemble_defect,complex_ensemble_defect,switch_single_strandedness,target_single_strandedness,complex_single_strandedness,valid_loop,loop_sequence,loop_structure,loop_energy
0,0.0,25.0,0,0,6946,GGGGAAGAGUCUAUUGAGCUAGUUGGUAGUCCUAUACAGAAACAGA...,AGGACTACCAACTAGCTCAATAGACTCTTC,AGGACTACCAACTAGCTCAATAGACTCTTC+GGGGAAGAGUCUAUU...,.((......)).......(((.....)))....................,((.........)).................,...........((...((....))......)).................,...........((...((....))......)).................,-2.147484e+07,-21474836.00,-2.147484e+07,0.464646,0.433333,0.457364,0.182700,0.144822,0.141583,0.0,0.0,0.0,True,ACAGAAACAGAGGA,..............,0.0
1,0.0,25.0,0,1,6805,GGGGAAGAGUCUAUUGAGCUAGUUGGUAGUCCUAUACAGAAACAGA...,TAAGCAGCCAGTATAGGAATAAAATGAAAC,TAAGCAGCCAGTATAGGAATAAAATGAAAC+GGGGAAGAGUCUAUU...,.((......)).......(((.....)))....................,...((.....))..................,....(..((....((....)).........)).)...............,....(..((....((....)).........)).)...............,-2.147484e+07,-21474836.00,-2.147484e+07,0.464646,0.333333,0.434109,0.182700,0.139911,0.147485,0.0,0.0,0.0,True,ACAGAAACAGAGGA,..............,0.0
2,0.0,25.0,0,2,6804,GGGGAAGAGUCUAUUGAGCUAGUUGGUAGUCCUAUACAGAAACAGA...,ATAAGCAGCCAGTATAGGAATAAAATGAAA,ATAAGCAGCCAGTATAGGAATAAAATGAAA+GGGGAAGAGUCUAUU...,.((......)).......(((.....)))....................,....((.....)).................,.....(..((....((....))........)).)...............,.....(..((....((....))........)).)...............,-2.147484e+07,-21474836.00,-2.147484e+07,0.464646,0.300000,0.426357,0.182700,0.139713,0.147343,0.0,0.0,0.0,True,ACAGAAACAGAGGA,..............,0.0
3,0.0,25.0,0,3,6771,GGGGAAGAGUCUAUUGAGCUAGUUGGUAGUCCUAUACAGAAACAGA...,TAGCTATATGGTAACAATCTCTCAATTGTC,TAGCTATATGGTAACAATCTCTCAATTGTC+GGGGAAGAGUCUAUU...,.((......)).......(((.....)))....................,...((....))...................,...((....((.......))..........))(((....))).......,...((....((.......))..........))(((....))).......,-2.147484e+07,-21474836.00,-2.147484e+07,0.464646,0.333333,0.434109,0.182700,0.158247,0.182734,0.0,0.0,0.0,True,ACAGAAACAGAGGA,..............,0.0
4,0.0,25.0,0,4,6712,GGGGAAGAGUCUAUUGAGCUAGUUGGUAGUCCUAUACAGAAACAGA...,GTATAAACCCAACAAAGGGATATATAATAG,GTATAAACCCAACAAAGGGATATATAATAG+GGGGAAGAGUCUAUU...,.((......)).......(((.....)))....................,....(..((.......))..).........,.......(((.........((.....))..)).)...............,.......(((.........((.....))..)).)...............,-2.147484e+07,-21474836.00,-2.147484e+07,0.464646,0.300000,0.426357,0.182700,0.209101,0.150506,0.0,0.0,0.0,True,ACAGAAACAGAGGA,..............,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13867,1.0,25.0,33,29,6467,GGGAGAACCAGUAAAAGAUAUCAUGAUUAAAUCAUACAGAAACAGA...,TAATCATGATATCTTTTACTGGTTCTGTGA,TAATCATGATATCTTTTACTGGTTCTGTGA+GGGAGAACCAGUAAA...,((.....)).((...((.(((((((((((.(((((((............,...(((((..(((.......)))..))))),((((((((((((((((((((((((((.(...).)))))))))))))...,((((((((((((((((((((((((((.(...).)))))))))))))...,-3.244000e+01,-5.23,-6.195000e+01,0.383838,0.300000,0.364341,0.086003,0.081594,0.085506,0.0,0.0,0.0,True,ACAGAAACAGAGGAGAU,((............).),0.0
13868,1.0,25.0,33,30,6466,GGGAGAACCAGUAAAAGAUAUCAUGAUUAAAUCAUACAGAAACAGA...,TTAATCATGATATCTTTTACTGGTTCTGTG,TTAATCATGATATCTTTTACTGGTTCTGTG+GGGAGAACCAGUAAA...,((.....)).((...((.(((((((((((.(((((((............,..((((((((......))).))))).....,(((((((((((((((((((((((((((.(..).)))))))))))))...,(((((((((((((((((((((((((((.(..).)))))))))))))...,-3.244000e+01,-3.47,-6.307000e+01,0.383838,0.300000,0.364341,0.086003,0.264900,0.086112,0.0,0.0,0.0,True,ACAGAAACAGAGGAGAU,((............).),0.0
13869,1.0,25.0,33,31,6465,GGGAG

In [1643]:
#display(testing)

In [1644]:
results_sorted = results.sort_values(by = ['complex_energy'])
#display(results_sorted)

In [1645]:
# html = test.to_html()

# text_file = open('index.html','w')
# text_file.write(html)
# text_file.close()

In [1646]:
#test.to_csv('nupack_data.csv')
results.to_csv('large_vienna_data.csv')
results.to_csv('large_tab_sep_vienna.csv', sep='\t')

In [1647]:
# stream = open('large_YAML_vienna.yaml', 'w')
# yaml.dump(results.to_dict(orient='records'), stream, sort_keys=False)
# yaml_data = yaml.dump(results.to_dict(orient='records'), sort_keys=False)
# #print(yaml_data)

# #test2.to_csv('test_data.csv')

In [1648]:
#For testing purposes
df1 = pd.read_csv('vienna_data - strange_data.csv', index_col=0)
#display(df1)
test_compare = df1[finished_data_names]
df2 = pd.read_csv('(0.01) Molarity-Coxsackievirus-Data.csv', index_col=0)
diff = df2.compare(test_compare)
display(diff)

switch_energy            target_energy       complex_energy             \
              self      other          self other           self      other   
0       -23.209999 -23.209999           NaN   NaN     -59.340000 -59.340000   
1       -23.209999 -23.209999         -0.17 -0.17     -33.639999 -33.639999   
2       -23.209999 -23.209999         -0.17 -0.17     -33.639999 -33.639999   
3       -23.209999 -23.209999         -0.17 -0.17     -26.629999 -26.629999   
4       -23.209999 -23.209999         -3.32 -3.32     -27.540001 -27.540001   
...            ...        ...           ...   ...            ...        ...   
1151           NaN        NaN         -0.52 -0.52     -43.320000 -43.320000   
1152           NaN        NaN           NaN   NaN     -44.110001 -44.110001   
1153           NaN        NaN           NaN   NaN     -44.610001 -44.610001   
1154           NaN        NaN           NaN   NaN     -45.160000 -45.160000   
1155           NaN        NaN           NaN   NaN     -48.950001 -48.950001   

     switch_gc           target_gc           complex_gc            \
          self     other      self     other       self     other   
0     0.464646  0.464646  0.433333  0.433333   0.457364  0.457364   
1     0.464646  0.464646  0.333333  0.333333   0.434109  0.434109   
2     0.464646  0.464646       NaN       NaN   0.426357  0.426357   
3     0.464646  0.464646  0.333333  0.333333   0.434109  0.434109   
4     0.464646  0.464646       NaN       NaN   0.426357  0.426357   
...        ...       ...       ...       ...        ...       ...   
1151  0.383838  0.383838       NaN       NaN   0.364341  0.364341   
1152  0.383838  0.383838       NaN       NaN   0.364341  0.364341   
1153  0.383838  0.383838  0.266667  0.266667   0.356589  0.356589   
1154  0.383838  0.383838  0.266667  0.266667   0.356589  0.356589   
1155  0.383838  0.383838  0.266667  0.266667   0.356589  0.356589   

     switch_ensemble_defect           target_ensemble_defect            \
                       self     other                   self     other   
0                  0.075309  0.075309               0.142913  0.142913   
1                  0.075309  0.075309               0.062859  0.062859   
2                  0.075309  0.075309               0.062800  0.062800   
3                  0.075309  0.075309               0.202906  0.202906   
4                  0.075309  0.075309               0.164807  0.164807   
...                     ...       ...                    ...       ...   
1151               0.057464  0.057464               0.265447  0.265447   
1152               0.057464  0.057464               0.268661  0.268661   
1153               0.057464  0.057464               0.259066  0.259066   
1154               0.057464  0.057464               0.256580  0.256580   
1155               0.057464  0.057464               0.254905  0.254905   

     complex_ensemble_defect            
                        self     other  
0                   0.064528  0.064528  
1                   0.032605  0.032605  
2                   0.032604  0.032604  
3                   0.249392  0.249392  
4                   0.095269  0.095269  
...                      ...       ...  
1151                0.083673  0.083673  
1152                0.084883  0.084883  
1153                0.080663  0.080663  
1154                0.076975  0.076975  
1155                0.070204  0.070204  

[1156 rows x 18 columns]

In [1649]:
# yep = pd.DataFrame()
# yep = pd.concat([yep,results])
# display(yep)

In [1650]:
# yepTest = []
# yepTest.extend(vArr)
# print(yepTest)
# yepTestData = pd.DataFrame(yepTest)
# display(yepTestData)